In [1]:
import numpy as np

In [2]:
def extended_euclidean(m, n):
  if n == 0:
    return 1, 0, m
  x, y, g = extended_euclidean(n, m % n)
  return y, x - (m // n)*y, g

def inv_mod(a, modulus):
  s, _, g = extended_euclidean(a, modulus)
  assert g == 1, ValueError('the modular inverse does not exist')
  return s % modulus

def RC4_K(key, n=256):
  S = np.array(range(n), dtype=int)
  j = 0
  for i in range(n):
    j = (j + S[i] + key[i % len(key)]) % n
    S[i], S[j] = S[j], S[i]
  return S

def RC4_PRGA(length, S, n=256):
  key_stream = np.zeros(length, dtype=int)
  i = 0
  j = 0
  for k in range(length):
    i = (i + 1) % n
    j = (j + S[i]) % n
    S[i], S[j] = S[j], S[i]
    key_stream[k] = S[(S[i] + S[j]) % n]
  return key_stream

def get_s_box(C):
  s_box = np.zeros(65536, dtype=int)
  for i in range(65536):
    s_box[i] = (C * inv_mod(i+1, 65537)) % 65537 - 1
  return s_box

In [3]:
def feistel_round(blocks, stream, s_box):
  newBlocks = blocks.copy()
  for i, block in enumerate(blocks):
    # Ln+1 = Rn
    newBlocks[i, :4] = block[4:]
    # Rn+1 = f(Rn, Kn) ^ Ln
    for j, (Ln, Rn, Kn) in enumerate(zip(block[:4], block[4:], stream[i])):
      newBlocks[i, 4+j] = s_box[Rn][Kn] ^ Ln
  return newBlocks

def feistel_network(plaintext, key, decode=False, debug=False):
  s_box = get_s_box(314159).reshape((256, 256)) % 256

  blocks = np.array([ord(char) for char in plaintext])
  # pad with zeros to have blocks.shape[0] % 8 == 0
  if (block_len := blocks.shape[0] % 8) != 0:
    blocks = np.append(blocks, (8 - block_len)*[0])
  # block size is 8 bytes
  blocks = blocks.reshape((blocks.shape[0] // 8, 8))

  # four rounds and four keys per round multiplied by len(blocks)
  stream = RC4_PRGA(16*blocks.shape[0], RC4_K(key)).reshape(4, blocks.shape[0], 4)

  # flip the stream if decoding
  if decode:
    stream = np.flip(stream, axis=0)

  if debug:
    print(f'Blocks: {blocks}')

  # four rounds
  for i in range(4):
    blocks = feistel_round(blocks, stream[i], s_box)
    if debug:
      print(f'Blocks: {blocks}')
      print(f'Stream: {stream[i]}')

  # swap L3 and R3 on each block
  swappedBlocks = blocks.copy()
  for i, block in enumerate(blocks):
    swappedBlocks[i, :4] = block[4:]
    swappedBlocks[i, 4:] = block[:4]

  if debug:
    print(f'Blocks: {swappedBlocks}')

  return ''.join([chr(num) for num in swappedBlocks.flatten()])

In [4]:
plaintext = 'Paul'*8
ciphertext = feistel_network(plaintext, [1,56,3])
decoded_plaintext = feistel_network(ciphertext, [1,56,3], decode=True)
print(f'Is decoded correctly: {plaintext == decoded_plaintext}')
print(f'Ciphertext: {ciphertext}')

Is decoded correctly: True
Ciphertext: 4ôÎÕ51BÝ?< ³ÉR³WßvTiq
